Notebook elaborado para apresentar os fundamentos da técnica de caminhamento elétrico utilizada no método geoelétrico.
Criado para a disciplina Geofísica II da Universidade Federal de Uberlândia.
O uso e a reprodução são livres para fins educacionais, pede-se apenas a citação da fonte.

[![License: GPL v3](https://img.shields.io/badge/License-GPLv3-blue.svg)](https://www.gnu.org/licenses/gpl-3.0)

Prof. Dr. Emerson Rodrigo Almeida<br>
Universidade Federal de Uberlândia<br>
Instituto de Geografia - Curso de Geologia<br>
[emerson.almeida@ufu.br](mailto:emerson.almeida@ufu.br)<br>

<p>&nbsp;</p>
Última modificação: 23/09/2022

# Instruções iniciais

Neste notebook você irá avaliar os resultados da inversão de pseudo-seções de resistividade aparente adquiridas em diferentes arranjos. Para isto **leia atentamente** as instruções apresentadas antes de cada célula e execute-as uma a uma, para acompanhar o processo corretamente.

Não se preocupe em compreender o código em si, **não é obrigatório que você conheça a programação** para obter e interpretar os resultados da demonstração. Mesmo assim, sinta-se livre caso queira perguntar qualquer coisa para aprender mais sobre o código e sua elaboração.

 
# Passo 01 - Preparação

Primeiramente vamos preparar o notebook com os pacotes, módulos e funções que precisaremos no decorrer da demonstração. A célula abaixo executa a instalação do pacote open source [SimPEG](https://docs.simpeg.xyz/), responsável por executar os cálculos relacionados à física do problema. Se você está executando este notebook no MyBinder ou no Google Colab não se preocupe, nada será instalado na sua máquina local. Execute esta célula uma vez antes de executar qualquer outra parte do código e não altere nada no código apresentado na célula para não criar problemas na execução do código.

Você pode executar o código na célula clicando sobre ela e em seguida clicando no botão **RUN** na barra do Jupyter Notebook ou pressionando as teclas **SHIFT+ENTER** no seu teclado.

In [ ]:
import modulos.demo_ce_mod as ce_mod
import modulos.demo_ce_inv as ce_inv

# Passo 02 - Configuração do modelo para simulação de dados adquiridos
Utilize as células abaixo para definir o comprimento e a profundidade totais do modelo. Ambos os valores devem ser positivos e informados em metros. Insira os valores desejados e execute a célula para defini-los.

In [ ]:
distancia_maxima = 14.0
profundidade_maxima = 3.0

A célula abaixo permite inserir alvos esféricos no modelo. Os alvos devem ser inseridos de acordo com o seguinte padrão:

**$'nome': [(x, -z, r), (rho)]$**

em que *'nome'* pode ser qualquer nome que você queira utilizar para um determinado alvo, *x* e *z* são as coordenadas do centro do alvo, em metros (note que z deve ser **negativo**), *'r'* é o raio do alvo e *rho* é a resistividade do alvo em Ohm.m. O meio onde os alvos serão inseridos tem, por padrão, a resistividade de 100 Ohm.m. Insira os valores numéricos na célula e execute-a para configurar os parâmetros.

**Importante:** Você pode inserir no modelo quantos alvos quiser, pois não há um número máximo. Cada conjunto de parâmetros referentes a um determinado alvo deve ser separado do conjunto anterior por uma vírgula, e deve todos os conjuntos devem ser colocados entre um par de chaves ( { } ), como no exemplo abaixo.

In [ ]:
alvos = {'alvo 1': [(5.0, -1.5, 0.5), (10.0)],     # [(x, z, r), (rho)]
         'alvo 2': [(9.0, -1.0, 0.5), (1000.0)],   # [(x, z, r), (rho)]
         'alvo 3': [(7.0, -0.5, 0.5), (300.0)]}    # [(x, z, r), (rho)]

# Passo 03 - Configuração do arranjo

Agora você irá definir os parâmetros do arranjo de eletrodos que irá simular a aquisição. Cada variável deve ser configurada da seguinte forma:
* **configuracao** : define o tipo de arranjo a ser utilizado. Utilize 'DD' para arranjo dipolo-dipolo, 'PD' para arranjo polo-dipolo e 'PP' para arranjo polo-polo;
* **espacamento** : define a distância de separação (em metros) entre cada eletrodo;
* **n_niveis** : define o número de níveis da investigação;
* **posicao_a** : define a posição do primeiro eletrodo A do arranjo (em metros), no início do caminhamento;
* **posicao_n** : define a posição do último eletrodo N do arranjo (em metros), no final do caminhamento.

Defina os valores desejados para estas variáveis na célula abaixo, executando-a em seguida para executar a simulação.

In [ ]:
configuracao = 'DD'
espacamento = 1.0
n_niveis = 10
posicao_a = 0.0
posicao_n = 14.0

# Passo 04 - Modelagem

Execute a célula abaixo para efetuar o cálculo da modelagem e obter os dados calculados.

In [ ]:
ce_mod.roda_demo(lx=distancia_maxima, lz=profundidade_maxima, tipo=configuracao, dx=espacamento, \
                 n=n_niveis, eletrodo_a=posicao_a, eletrodo_n=posicao_n, alvos=alvos, arquivo='ce_temp')

# Passo 05 - Configuração dos parâmetros do modelo da inversão
Utilize a célula abaixo para definir o comprimento e a profundidade totais do modelo que você deseja avaliar. Ambos os valores devem ser positivos e informados em metros. Insira os valores desejados e execute a célula para defini-los.

In [ ]:
distancia_maxima_inv = 14.0
profundidade_maxima_inv = 3.0

Aqui será necessário definir uma **discretização** para o modelo, i.e., será necessário subdividi-lo em células de forma que a resistividade cada célula representa uma incógnita no conjunto de equações que define o problema inverso. Quanto menor for o tamanho da célula, maior será a suavidade das variações de resistividade calculadas, porém mais demorado será o cálculo pois haverá mais incógnitas para serem calculadas.

Defina o tamnho da célula (em metros) que você deseja aplicar ao modelo na célula abaixo, executando-a em seguida para configurar este valor.

In [ ]:
dh = 0.1

O cálculo da inversão requer que seja inserido um modelo inicial que será adotado como ponto de partida para a busca da solução ideal. É possível incluir diferentes alvos e camadas no modelo inicial, porém para simplificar o processo o modelo inicial será definido como um semi-espaço homogêneo.

Defina um valor para a resistividade do semi-espaço homogêneo (em Ohm.m) na célula abaixo, executando-a para configurar este valor.

In [ ]:
resistividade_bg = 100.0

# Passo 06 - Definição dos parâmetros da inversão

É necessário também definir certos parâmetros para que o cálculo da inversão seja realizado corretamente. É possível alterar diversos parâmetros neste processo, porém aqui serão definidos apenas os seguintes parâmetros essenciais:

* **n_max_iteracoes** : define o número máximo de iterações que o programa irá executar em busca da melhor solução matemática;
* **chi** = define a norma do erro mínimo aceitável para o ajuste entre os dados observados e os dados preditos.

Defina os parâmetros na célula abaixo e execute-a para configurá-los.

In [ ]:
n_max_iteracoes = 40
chi = 1

# Passo 07 - Inversão

Execute a célula abaixo para executar a inversão e obter o modelo de resistividade para análise. **Atenção!** Este processo pode demorar alguns minutos, dependendo dos parâmetros que você configurou. Aguarde e não execute nada neste notebook até que as figuras apareçam.

In [ ]:
ce_inv.roda_demo('ce_temp_volt.dat', 'ce_temp_volt.top', (distancia_maxima_inv, profundidade_maxima_inv), \
                 resistividade_bg, n_iter=n_max_iteracoes, chifact=chi, dh=dh)

# Tópicos para reflexão

Modifique os parâmetros inseridos nas células acima e reflita sobre as seguintes questões:

* Como o tipo de arranjo influencia no modelo obtido após a inversão?
* O que acontece com os alvos obtidos na inversão quando se muda o espaçamento entre eletrodos?
* O que se observa em relação ao tamanho dos alvos quando são comparados o modelo original em que você inseriu os alvos e o modelo obtido após a inversão?